In [1]:
import numpy as np 


In [2]:
X = np.load("data/processed/X_hand_open.npy")
y = np.load("data/processed/y_hand_open.npy")

print("X shape:", X.shape)   # (windows, samples, channels)
print("y shape:", y.shape)


X shape: (578, 200, 5)
y shape: (578,)


In [3]:
# rms how strong is the muscle activation, emphasizes peaks more 
# mav average muscle activity, more linear compared to RMS
# WL how much the wave wiggles, more motor units firing 
# More motor units firing => more oscillaions => higher WL. Smooth / steady contractions -> lower WL
# ZC (zero crossing) - How often does the signal change direction?
# more zero crossings => higher frequency activity, whilst fewer => slower activiation
# SSC = slope sign changes 

def rms(x):
    return np.sqrt(np.mean(x**2))

def mav(x):
    return np.mean(np.abs(x))

def wl(x):
    return np.sum(np.abs(np.diff(x)))

def zc(x, threshold=0.01):
    return np.sum(
        (x[:-1] * x[1:] < 0) &
        (np.abs(x[:-1] - x[1:]) >= threshold)
    )

def ssc(x, threshold=0.01):
    return np.sum(
        ((x[1:-1] - x[:-2]) * (x[1:-1] - x[2:]) > threshold)
    )


In [4]:
def extract_features(X):
    num_windows, _, num_channels = X.shape
    features = []

    for w in range(num_windows):
        feat_window = []
        for ch in range(num_channels):
            sig = X[w, :, ch]
            feat_window.extend([
                rms(sig),
                mav(sig),
                wl(sig),
                zc(sig),
                ssc(sig)
            ])
        features.append(feat_window)

    return np.array(features)


In [7]:
X_feat = extract_features(X)

print("Feature matrix shape:", X_feat.shape)
X_feat = np.nan_to_num(X_feat, nan=0.0)



Feature matrix shape: (578, 25)


In [8]:
# sanity check to see any NaN

print("Any NaNs?", np.isnan(X_feat).any())

print("Feature means (first 5):", X_feat.mean(axis=0)[:5])


Any NaNs? False
Feature means (first 5): [2.71746056e-02 1.97135119e-02 2.83113146e+00 1.97595156e+01
 1.29757785e-01]


In [9]:
# how many NaN's did I have 
np.sum(np.isnan(X_feat))


np.int64(0)

In [10]:
np.save("data/processed/X_hand_open_features.npy", X_feat)
np.save("data/processed/y_hand_open_features.npy", y)
